In [2]:
import pandas as pd 
import numpy as np 
import torch
import seaborn as sns 
import matplotlib.pyplot as plt 
import nltk 
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words') 
nltk.download('vader_lexicon')
plt.style.use('ggplot')

ImportError: Unable to import required dependencies:
numpy: No module named 'numpy'

In [ ]:
#read and clean data 

df=pd.read_csv(r"roberta_scores.csv")
print(df.shape)
df

In [ ]:
ax=df['Score'].value_counts().sort_index().plot(kind='bar',title='cocont of reviews',figsize=(10,5))
ax.set_xlabel('Review Stars')
plt.show()

In [ ]:
#example

example = df['Text'][50]
print(example)

In [ ]:
#nltk
tokens=nltk.word_tokenize(example)
tagged=nltk.pos_tag(tokens)
tagged

In [ ]:
entities=nltk.chunk.ne_chunk(tagged)
entities.pprint()

$$VADER (Valence Aware Dictionary and sEntiment Reasoner)$$

In [ ]:
sia=SentimentIntensityAnalyzer()
sia.polarity_scores(example)

<h4>Run On full DF<h4>

In [ ]:
res={}
for i,row in tqdm(df.iterrows(),total=len(df)):
    text = row['Text']
    myid= row['Id']
    res[myid]=sia.polarity_scores(text)

In [ ]:
vaders=pd.DataFrame(res).T
vaders= vaders.reset_index().rename(columns={'index':'Id'})
df_vaders= df.merge(vaders,how="right")
df_vaders

In [ ]:
#plot df_vaders
fig, axs  = plt.subplots(1,3,figsize=(12,5))
sns.barplot(data=df_vaders,x="Score",y="pos",ax=axs[0])
sns.barplot(data=df_vaders,x="Score",y="neg",ax=axs[1])
sns.barplot(data=df_vaders,x="Score",y="neu",ax=axs[2])
axs[0].set_title("Positive")
axs[1].set_title("Negative")
axs[2].set_title("Neutral")
plt.show()

$$Roberta Pretrained Model$$

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification 
from scipy.special import softmax  


In [ ]:
Model = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer= AutoTokenizer.from_pretrained(Model)
model = AutoModelForSequenceClassification.from_pretrained(Model)


In [ ]:
#vader results example 
encoded_text= tokenizer(example,return_tensors='pt')
output=model(**encoded_text)
scores= output[0][0].detach().numpy()
scores